# Ensembles: Extremely Randomized Trees

Random Forest es uno de los modelos más usados en la actualidad y suele brindar excelentes resultados.

### Fundamento de Extremely Randomized Trees:

En este caso la idea es la de agregar un elemento más de aleatoriedad. Seguramente recuerda que en la materia Introducción a la Inteligencia Artificial tuvo que hacer un gran esfuerzo cuando le pedimos que resolviera "manualmente" los primeros árboles. El esfuerzo intelectual se dedicaba a seleccionar con qué variable era conveniente comenzar la división del árbol lo cual implicaba ver con qué variable "clasificábamos" la mayor cantidad de casos, luego aprendimos que podíamos sistematizarlo, muy bien con los Árboles Extremadamente Aleatorizados **la selección de esta variable en cada nodo también ser hará al azar** junto con el resto de las variables que ya se "aleatorizaban" en Random Forest!


### Extremely Randomized Trees en Scikit-Learn:

Encontrará la documentación para el clasificador aquí: https://scikit-learn.org/stable/modules/ensemble.html#forests-of-randomized-trees



Para Clasificación la documentación: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesClassifier.html#sklearn.ensemble.ExtraTreesClassifier

La sintaxis es la siguiente:

~~~
sklearn.ensemble.ExtraTreesClassifier(n_estimators=100, *, criterion='gini', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=False, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0.0, max_samples=None)
~~~

Para Rregresión la documentación es: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.ExtraTreesRegressor.html#sklearn.ensemble.ExtraTreesRegressor

La sintaxis es la siguiente:
~~~
sklearn.ensemble.ExtraTreesRegressor(n_estimators=100, *, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=False, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)

~~~


Los principales son, como podrá observar la mayoría son los mismos que para Random Forest.

- **n_estimators=100** Es la cantidad de árboles que formarán el ensemble (forest)  
- **max_features='auto'**  Si se pasa un número entero, utilizará dicha cantidad, si se pasa un float, entonces se lo considerará como la fracción del total de features. En auto tomará la raíz cuadrada de todas las features. Si se pasa None, entonces utilizará todas las variables disponibles. 
- **bootstrap=True**  Si se pasa True, usa bootstrap para generar las observaciones, sino utiliza todas las observaciones.
- **max_samples=None** cuando bootstrap = True indica la cantidad de observaciones para cada árbol. Si se pasa None, entonces utiliza la misma cantidad de observaciones que las originales (aunque no necesariamente son las mismas!), si se pasa un entero, toma esa cantidad de observaciones, si se pasa un float, lo toma como fracción del total de observaciones.



### Ejemplo

Repitamos el problema de la clasificación de los vinos blancos:

#### Los datos:

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('data/winequality-white.csv', sep=';')
X=df.drop(axis=1,columns='quality')
y=df['quality']


Por las dudas vamos a desordenar un poco lo datos con Shuffle, porque me parece que siguen cierto patrón. 

In [ ]:
from sklearn.utils import shuffle
X,y=shuffle(X,y, random_state=123)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=123, stratify=y)

### Creamos el modelo de Random Forest

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etc=ExtraTreesClassifier(n_estimators=100,n_jobs=-1,random_state=123)

### Entrenamos con .fit

In [ ]:
etc.fit(X_train, y_train)

ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=123, verbose=0,
                     warm_start=False)

### Evaluemos Accuracy con Train - Test (Hold Out)

In [ ]:
etc.score(X_test,y_test)

0.6979591836734694

### Evaluemos con Cross Validation

In [ ]:
from sklearn.model_selection import cross_val_score 


In [ ]:
AC_scores = cross_val_score(etc, X, y, cv=5)
AC_media=AC_scores.mean()
AC_desvio=AC_scores.std()
# veamos los resultados
print("AC en cada fold: ",AC_scores)
print(AC_media,' x/- ',AC_desvio)

AC en cada fold:  [0.68571429 0.66836735 0.67142857 0.67824311 0.6741573 ]
0.6755821225323633  x/-  0.0060179172947692925


### Con GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

etc1=ExtraTreesClassifier(n_jobs=-1,random_state=123)
grid_etc1= GridSearchCV(estimator=etc1, param_grid={'n_estimators':[100,150,200,250,300,400,500]}, cv=5,n_jobs=-1, return_train_score=True)


In [ ]:
grid_etc1.fit(X,y)

GridSearchCV(cv=5, error_score=nan,
             estimator=ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0,
                                            class_weight=None, criterion='gini',
                                            max_depth=None, max_features='auto',
                                            max_leaf_nodes=None,
                                            max_samples=None,
                                            min_impurity_decrease=0.0,
                                            min_impurity_split=None,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100, n_jobs=-1,
                                            oob_score=False, random_state=123,
                                            verbose=0, warm_start=False),
             iid='deprecated',

In [ ]:
AC=grid_etc1.best_score_
AC

0.6855858747993578

In [ ]:
grid_etc1.best_params_

{'n_estimators': 300}

### Conclusión

> El valor es muy cercano al obtenido con Random Forest. 

### Ejercicio:

Probar con GridSearchCV variando otros hiperparámetros de Extra Trees Classifier, como por ejemplo max_features y max_samples, usar por lo menos dos o 3 valores para cada uno. 